# Aula 1 - Criando um agente de funções

## Vídeo 1.2 - Configurando o LlamaIndex

In [ ]:
!pip install llama-index llama_index.embeddings.huggingface llama-index-readers-file llama-index-llms-groq arxiv llama-index-tools-tavily-research crewai crewai-tools gradio

In [ ]:
from llama_index.core import Settings

In [ ]:
from google.colab import userdata
groq = userdata.get('GROQ_API_KEY')

In [ ]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-groq-70b-8192-tool-use-preview", api_key=groq)

In [ ]:
Settings.llm = llm

In [ ]:
def calcular_engajamento(curtidas: int, comentarios: int, compartilhamentos: int, seguidores: int) -> str:
    """
    Calcula o engajamento total e a taxa de engajamento de uma postagem.

    Args:
        curtidas (int): Número de curtidas na postagem.
        comentarios (int): Número de comentários na postagem.
        compartilhamentos (int): Número de compartilhamentos da postagem.
        seguidores (int): Número total de seguidores.

    Returns:
        dict: Um dicionário contendo o engajamento total (int) e a taxa de engajamento (float, em porcentagem).
    """
    engajamento_total = curtidas + comentarios + compartilhamentos
    taxa_engajamento = (engajamento_total / seguidores) * 100 if seguidores > 0 else 0
    dicionario = {"engajamento_total": engajamento_total,"taxa_engajamento": round(taxa_engajamento, 2)}
    resultado = f"O engajamento total é {engajamento_total} e a taxa de engajamento é {round(taxa_engajamento, 2)}%."
    # Return the dictionary directly
    return resultado

## Vídeo 1.3 - Transformando a função em ferramenta

In [ ]:
from llama_index.core.tools import FunctionTool

In [ ]:
ferramenta = FunctionTool.from_defaults(fn=calcular_engajamento,name="Calcular Engajamento",description=(
        "Calcula o engajamento total e a taxa de engajamento de uma postagem nas redes sociais. "
        "Argumentos: curtidas (int), comentarios (int), compartilhamentos (int), seguidores (int). "
        "Retorna uma string com o engajamento total e a taxa de engajamento."
    ))

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm,
)

In [ ]:
from llama_index.core.agent import AgentRunner

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response =agent.chat("Qual é o engajamento de uma postagem que teve 150 curtidas, 35 comentários, 20 compartilhamentos, e o perfil tem 2000 seguidores?")

In [ ]:
response = agent.chat("Quem é Albert Einstein?")

## Vídeo 1.4 - Consultando artigos

In [ ]:
import arxiv

In [ ]:
def consulta_artigos(titulo: str) -> str:
    """
    Consulta os artigos na base de dados arXiv e retorna resultados formatados.
    """
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = []
    for resultado in busca.results():
        resultados.append(f"Título: {resultado.title}\n"
                          f"Resumo: {resultado.summary}\n"
                          f"Categoria: {resultado.primary_category}\n"
                          f"Link: {resultado.entry_id}\n")

    return "\n\n".join(resultados)

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("Me retorne artigos sobre o uso da inteligência artificial nas redes sociais")

# Aula 2 - Aprofundando nas pesquisas

## Vídeo 2.1 - Adotando ferramentas prontas

In [ ]:
tavily_key = userdata.get('tavily')

In [ ]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key,
)

In [ ]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

In [ ]:
tavily_tool.search("Me retorne artigos científicos sobre o uso da inteligência artificial nas redes sociais que você encontrar na web", max_results=3)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(tavily_tool.to_tool_list())

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("Me retorne artigos científicos sobre o uso da inteligência artificial nas redes sociais que você encontrar na web")

In [ ]:
print(response)

## Vídeo 2.2 - Criando uma base de dados vetorial

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [ ]:
url = '/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/artigo1.pdf'
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
url = '/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/livro1.pdf'
livro = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")

In [ ]:
artigo_index = VectorStoreIndex.from_documents(artigo)
livro_index = VectorStoreIndex.from_documents(livro)

In [ ]:
artigo_index.storage_context.persist(persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/artigo")
livro_index.storage_context.persist(persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/livro")

## Vídeo 2.3 - Criando engines de busca

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

In [ ]:
storage_context = StorageContext.from_defaults(
        persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/artigo"
    )
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
        persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/livro"
    )
livro_index = load_index_from_storage(storage_context)

In [ ]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3)
livro_engine = livro_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre algoritmos de inteligência artificial nas redes sociais. "
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=livro_engine,
        metadata=ToolMetadata(
            name="livro_engine",
            description=(
                "Fornece informações sobre avanços e tendências sobre inteligência artificial. "
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            ),
        ),
    ),
]

# Aula 3 - Verificando o funcionamento do agente

## Vídeo 3.1 - Consultando os textos com agentes

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent_document = AgentRunner(agent_worker)

In [ ]:
response = agent_document.chat(
    "Quais os principais algoritmos de IA usados nas redes sociais?"
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent_document = AgentRunner(agent_worker)

In [ ]:
response = agent_document.chat(
    "Quais as principais tendências de IA que eu deveria estudar?"
)

## Vídeo 3.2 - Usando um agente ReAct

In [ ]:
from llama_index.core.agent import ReActAgent

In [ ]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
)

In [ ]:
response = agente.chat("Quais os principais algoritmos de IA usados nas redes sociais?")

In [ ]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
)

In [ ]:
response = agente.chat("Quais as principais tendências de IA que eu deveria estudar?")

In [ ]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
)

In [ ]:
response = agente.chat("Como o feed de uma rede social utiliza IA?")

## Vídeo 3.3 - Configurando o CrewAI

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import LlamaIndexTool
import arxiv

In [ ]:
ferramenta_artigos = FunctionTool.from_defaults(fn=consulta_artigos, name="consulta_artigos", description="Consulta os artigos na base de dados arXiv e retorna resultados formatados.")
tool = LlamaIndexTool.from_tool(ferramenta_artigos)

In [ ]:
nvidia = userdata.get('nvidia-llama405b')

In [ ]:
llm = LLM(
    model="nvidia_nim/meta/llama-3.3-70b-instruct",
    api_key=nvidia
)

In [ ]:
agent = Agent(
    role='Agente de pesquisa',
    goal='Fornece artigos científicos sobre um assunto de interesse.',
    backstory='Um agente expert em pesquisa científica que possui a habilidade de buscar artigos no arxiv',
    tools=[tool],
    llm=llm
)

In [ ]:
task1 = Task(
    description="Busque artigos científicos sobre o uso da inteligência artificial nas redes sociais. ",
    expected_output="5 artigos e seus respectivos links",
    agent=agent
)

In [ ]:
crew = Crew(
    agents=[agent],
    tasks=[task1],
    verbose=1,
)

In [ ]:
result = crew.kickoff()

print("######################")
print(result)

# Aula 4 - Múltiplas tarefas e agentes

## Vídeo 4.1 - Adicionando uma nova tarefa

In [ ]:
import requests

def baixar_pdf_arxiv(link):
    """
    Baixa o PDF de um artigo do arXiv dado o link do artigo.

    Args:
        link (str): O link para o artigo no arXiv.

    Returns:
        str: O caminho do arquivo salvo ou uma mensagem de erro.
    """
    try:
        # Verifica se o link é do arXiv
        if "arxiv.org" not in link:
            return "O link fornecido não é um link válido do arXiv."

        # Extrai o ID do artigo do link
        artigo_id = link.split("/")[-1]

        # Monta o link direto para o PDF
        pdf_url = f"https://arxiv.org/pdf/{artigo_id}.pdf"

        # Faz o download do PDF
        response = requests.get(pdf_url, stream=True)
        if response.status_code == 200:
            nome_arquivo = f"artigo_{artigo_id}.pdf"
            with open(nome_arquivo, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024):
                    f.write(chunk)
            return f"PDF salvo como {nome_arquivo}"
        else:
            return f"Erro ao baixar o PDF. Código de status: {response.status_code}"
    except Exception as e:
        return f"Ocorreu um erro: {e}"

In [ ]:
ferramenta_baixar = FunctionTool.from_defaults(fn=baixar_pdf_arxiv, name="baixa_artigos", description="Baixa o PDF de um artigo do arXiv dado o link do artigo.")
tool_baixar = LlamaIndexTool.from_tool(ferramenta_baixar)

In [ ]:
agent = Agent(
    role='Agente de pesquisa',
    goal='Fornece artigos científicos sobre um assunto de interesse.',
    backstory='Um agente expert em pesquisa científica que possui a habilidade de acessar e baixar artigos no arxiv',
    tools=[tool,tool_baixar],
    llm=llm
)

In [ ]:
task2 = Task(
    description="""Usando os links gerados na primeira tarefa, baixe os artigos em PDF.""",
    expected_output="Nome dos artigos baixados",
    agent=agent,
)

In [ ]:
crew2 = Crew(
    agents=[agent],
    tasks=[task1, task2],
    verbose=1,
)

In [ ]:
# Get your crew to work!
result = crew2.kickoff()

print("######################")
print(result)

## Vídeo 4.2 - Combinando agentes

In [ ]:
tavily_tools = tavily_tool.to_tool_list()
tools = [LlamaIndexTool.from_tool(t) for t in tavily_tools]

In [ ]:
agent_web = Agent(
    role='Agente de pesquisa por documentos na web',
    goal='Fornece artigos científicos encontrados na web sobre um assunto de interesse.',
    backstory='Um agente expert em pesquisa científica que possui a habilidade de buscar artigos na web e retorna artigos de fontes como springer, scielo, reseachgate entre outros',
    tools=[*tools],
    llm=llm
)

In [ ]:
agente_verificacao = Agent(
    role='Agente de pesquisa que verifica se os documentos encontrados na web são realmente artigos científicos ou artigos em outros formatos',
    goal='Fornece como saída os artigos científicos, os artigos em outros formatos, de fontes como o medium ou outros blogs, devem ser excluídos e substituídos por artigos científicos',
    backstory='Um agente expert em pesquisa científica',
    tools=[*tools],
    llm=llm
)

In [ ]:
task1 = Task(
    description="Busque artigos científicos sobre o uso da inteligência artificial nas redes sociais. ",
    expected_output="5 artigos e seus respectivos links",
    agent=agent_web
)

In [ ]:
task2 = Task(
    description="Verifique se os artigos encontrados na web realmente são artigos científicos. ",
    expected_output="5 artigos e seus respectivos links",
    agent=agente_verificacao
)

In [ ]:
crew3 = Crew(
    agents=[agent_web,agente_verificacao],
    tasks=[task1, task2],
    verbose=1,  # You can set it to 1 or 2 to different logging levels
)

In [ ]:
result = crew3.kickoff()

print("######################")
print(result)

## Vídeo 4.3 - Trabalhando hierarquia

In [ ]:
task = Task(
    description="Busque artigos científicos no arxiv sobre o uso da inteligência artificial nas redes sociais. ",
    expected_output="5 artigos e seus respectivos links",
    agent=agent
)

In [ ]:
manager = Agent(
    role="Gerente do projeto",
    goal="Gerenciar a equipe com eficiência e garantir a conclusão de tarefas de alta qualidade",
    backstory="Você é um gerente de projeto experiente, habilidoso em supervisionar projetos complexos e guiar equipes para o sucesso. Sua função é coordenar os esforços dos membros da equipe, garantindo que cada tarefa seja concluída no prazo e no mais alto padrão.",
    allow_delegation=True,
    llm=llm

)

In [ ]:
crew_hierarquica = Crew(
    agents=[agent, agent_web,agente_verificacao],
    tasks=[task,task1, task2],
    manager_agent=manager,
    process=Process.hierarchical,
    verbose=1,
)

In [ ]:
result = crew_hierarquica.kickoff()

print("######################")
print(result)

# Aula 5 - Criando uma aplicação

## Vídeo 5.1 - Procurando sobre qualquer tema

In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew, Process

def pesquisar_artigos(tema):
    # Criação dos agentes com o tema dinâmico
    agent = Agent(
        role='Agente de pesquisa',
        goal='Fornece artigos científicos sobre um assunto de interesse.',
        backstory='Um agente expert em pesquisa científica que possui a habilidade de acessar e baixar artigos no arxiv',
        tools=[tool,tool_baixar],
        llm=llm
    )

    agent_web = Agent(
        role='Agente de pesquisa por documentos na web',
        goal='Fornece artigos científicos encontrados na web sobre um assunto de interesse.',
        backstory='Um agente expert em pesquisa científica que possui a habilidade de buscar artigos na web',
        tools=[*tools],
        llm=llm
    )

    agente_verificacao = Agent(
        role='Agente de pesquisa que verifica documentos',
        goal='Fornece como saída apenas artigos científicos válidos',
        backstory='Um agente expert em pesquisa científica',
        tools=[*tools],
        llm=llm
    )

    manager = Agent(
        role="Gerente do projeto",
        goal="Gerenciar a equipe com eficiência",
        backstory="Gerente de projeto experiente que coordena os esforços da equipe",
        allow_delegation=True,
        llm=llm
    )

    # Criação das tasks usando o tema recebido como parâmetro
    task = Task(
        description=f"Busque artigos científicos no arxiv sobre {tema}.",
        expected_output="5 artigos e seus respectivos links",
        agent=agent
    )

    task1 = Task(
        description=f"Busque artigos científicos sobre {tema}.",
        expected_output="5 artigos e seus respectivos links",
        agent=agent_web
    )

    task2 = Task(
        description="Verifique se os artigos encontrados na web realmente são artigos científicos.",
        expected_output="5 artigos e seus respectivos links",
        agent=agente_verificacao
    )

    crew_hierarquica = Crew(
        agents=[agent, agent_web, agente_verificacao],
        tasks=[task, task1, task2],
        manager_agent=manager,
        process=Process.hierarchical,
        verbose=1,
    )

    result = crew_hierarquica.kickoff()
    return result

# Interface Gradio
interface = gr.Interface(
    fn=pesquisar_artigos,
    inputs=gr.Textbox(label="Digite o tema para pesquisa"),
    outputs=gr.Textbox(label="Resultados"),
    title="👨‍🔬👩‍🔬 Pesquisador de Artigos Científicos",
    description="Digite um tema para encontrar artigos científicos relacionados.",
    theme=gr.themes.Glass()
)

# Iniciar a aplicação
interface.launch()


## Vídeo 5.2 - Perguntando sobre documentos diversos.

In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew, Process
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent

storage_context = StorageContext.from_defaults(
        persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/artigo"
    )
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
        persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/livro"
    )
livro_index = load_index_from_storage(storage_context)


artigo_engine = artigo_index.as_query_engine(similarity_top_k=3)
livro_engine = livro_index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre algoritmos de inteligência artificial nas redes sociais. "
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=livro_engine,
        metadata=ToolMetadata(
            name="livro_engine",
            description=(
                "Fornece informações sobre avanços e tendências sobre inteligência artificial. "
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            ),
        ),
    ),
]

def pesquisar_artigos(tema):
    # Criação dos agentes com o tema dinâmico
    agent = Agent(
        role='Agente de pesquisa',
        goal='Fornece artigos científicos sobre um assunto de interesse.',
        backstory='Um agente expert em pesquisa científica que possui a habilidade de acessar e baixar artigos no arxiv',
        tools=[tool, tool_baixar],
        llm=llm
    )

    agent_web = Agent(
        role='Agente de pesquisa por documentos na web',
        goal='Fornece artigos científicos encontrados na web sobre um assunto de interesse.',
        backstory='Um agente expert em pesquisa científica que possui a habilidade de buscar artigos na web',
        tools=[*tools],
        llm=llm
    )

    agente_verificacao = Agent(
        role='Agente de pesquisa que verifica documentos',
        goal='Fornece como saída apenas artigos científicos válidos',
        backstory='Um agente expert em pesquisa científica',
        tools=[*tools],
        llm=llm
    )

    manager = Agent(
        role="Gerente do projeto",
        goal="Gerenciar a equipe com eficiência",
        backstory="Gerente de projeto experiente que coordena os esforços da equipe",
        allow_delegation=True,
        llm=llm
    )

    # Criação das tasks usando o tema recebido como parâmetro
    task = Task(
        description=f"Busque artigos científicos no arxiv sobre {tema}.",
        expected_output="5 artigos e seus respectivos links",
        agent=agent
    )

    task1 = Task(
        description=f"Busque artigos científicos sobre {tema}.",
        expected_output="5 artigos e seus respectivos links",
        agent=agent_web
    )

    task2 = Task(
        description="Verifique se os artigos encontrados na web realmente são artigos científicos.",
        expected_output="5 artigos e seus respectivos links",
        agent=agente_verificacao
    )

    crew_hierarquica = Crew(
        agents=[agent, agent_web, agente_verificacao],
        tasks=[task, task1, task2],
        manager_agent=manager,
        process=Process.hierarchical,
        verbose=1,
    )

    result = crew_hierarquica.kickoff()
    return result

# Função para a segunda aba
def pergunta(query):
    agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    )
    response = agente.chat("Quais os principais algoritmos de IA usados nas redes sociais?")

    return response

# Interface Gradio para a primeira aba
pesquisa_interface = gr.Interface(
    fn=pesquisar_artigos,
    inputs=gr.Textbox(label="Digite o tema para pesquisa"),
    outputs=gr.Textbox(label="Resultados"),
    title="👨‍🔬👩‍🔬 Pesquisador de Artigos Científicos",
    description="Digite um tema para encontrar artigos científicos relacionados.",
    theme=gr.themes.Glass()
)

# Interface Gradio para a segunda aba
hello_interface = gr.Interface(
    fn=pergunta,
    inputs=gr.Textbox(label="Faça sua pergunta"),
    outputs=gr.Textbox(label="Saída"),
    title="Perguntas sobre IA nas redes sociais",
    description="Exemplo simples de uma segunda aba."
)

# Combinação das abas em um bloco de abas
with gr.Blocks(theme=gr.themes.Glass()) as app:
    with gr.Tab("Pesquisa de Artigos"):
        pesquisa_interface.render()
    with gr.Tab("Hello World"):
        hello_interface.render()

# Iniciar a aplicação
app.launch()
